Data Collection workflow for finding on chain fees from closures

In [ ]:
import numpy as np
import pandas as pd
import requests
import json

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
pubkey = "03d95924d6de5784f0b610d5a597d4a94e247c6da5e8db39a3d051cd9059467719"
index=0
df = pd.DataFrame()
url = "https://mempool.space/api/v1/lightning/channels?public_key="+pubkey+"&status=closed&index="+str(index)
closed_channels = pd.DataFrame(json.loads(requests.get(url).text))
while closed_channels.shape[0] != 0:
    df = pd.concat([df,closed_channels])
    index += 10
    url = "https://mempool.space/api/v1/lightning/channels?public_key="+pubkey+"&status=closed&index="+str(index)
    closed_channels = pd.DataFrame(json.loads(requests.get(url).text))
    if index % 100 == 0:
        print(f"Number of rows: {df.shape[0]}")
print(f"Completed with {df.shape[0]} channels pulled.")

In [ ]:
df.head()

In [ ]:
def get_close_tx(channel_id):
    chan_url = "https://mempool.space/api/v1/lightning/channels/" + str(channel_id)
    response = requests.get(chan_url)
    try:
        tx_id = json.loads(response.text)['closing_transaction_id']
        return tx_id
    except ValueError:
        print(f"Error with channel_id {channel_id}")
        return ""

In [ ]:
#df['closing_tx'] = df.apply(lambda row: get_close_tx(row['id']), axis=1)
df['closing_tx'] = df.progress_apply(lambda row: get_close_tx(row['id']), axis=1) #tqdm progress bar
print(f"Found all closing txs")

In [ ]:
df.head()

In [ ]:
mempool_api_url = "https://mempool.space" #or local mempool.space instance

In [ ]:
def get_sweep_transactions(closing_tx):
    lst = []
    for output in json.loads(closing_tx.text)['vout']:
        lst.append(output['scriptpubkey_address'])
    return lst

In [ ]:
def get_close_fee(tx_id, closing_reason):
    closing_transaction_url = mempool_api_url + "/api/tx/" + tx_id
    closing_tx = requests.get(closing_transaction_url) #If using local mempool instance, may have to add verify=False
    fee = json.loads(closing_tx.text)['fee']
    if closing_reason == 1:
        return fee
    lst_of_sweep_addresses = get_sweep_transactions(closing_tx)
    for sweep_address in lst_of_sweep_addresses:
        sweep_tx_url = mempool_api_url + "/api/address/" + sweep_address + "/txs"
        sweep_tx = requests.get(sweep_tx_url) #If using local mempool instance, may have to add verify=False
        fee += json.loads(sweep_tx.text)[0]['fee']
    return fee

In [ ]:
df['total_fee'] = 0
#df['total_fee'] = df.apply(lambda row: get_close_fee(row['closing_tx'],row['closing_reason']), axis=1)
df['total_fee'] = df.progress_apply(lambda row: get_close_fee(row['closing_tx'],row['closing_reason']), axis=1) #tqdm progress bar
df.to_csv('df_fee.csv',index=False)
print(f"Fees Completed")

In [ ]:
df.head()